In [3]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import sklearn
import seaborn as sns
from sklearn.ensemble import VotingClassifier

from steps.prepare_data import load_processed_data
from utils.model import predict, load_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_data, test_data = load_processed_data()

train_data_x = train_data.drop(columns=["target"])
train_data_y = train_data["target"]

test_data_x = test_data.drop(columns=["target"])
test_data_y = test_data["target"]

lightGbmV2_p069_r033_090auc, _ = load_model("LightGbmV2_p069_r033_090auc.pickle")
lightGbmV1_p041_r064_089auc, _ = load_model("LightGbmV1_p041_r064_089auc.pickle")
xgboostV1_p072_r031_090auc, _ = load_model("XgboostV1_p072_r031_090auc.pickle")

eclf1 = VotingClassifier(
    estimators=[
        ("lightGbmV2_p069_r033_090auc", lightGbmV2_p069_r033_090auc),
        ("lightGbmV1_p041_r064_089auc", lightGbmV1_p041_r064_089auc),
        ("xgboostV1_p072_r031_090auc", xgboostV1_p072_r031_090auc),
    ],
    voting="soft",
)

eclf1.fit(train_data_x, train_data_y)

test_data_preds = eclf1.predict(test_data_x)

NameError: name 'sklearn' is not defined

In [5]:
auc = sklearn.metrics.roc_auc_score(test_data_y, test_data_preds)
print(f"AUC: {auc:.2f}")

# classification report
class_report = sklearn.metrics.classification_report(test_data_y, test_data_preds)
print("Classification Report:")
print(class_report)

AUC: 0.71
Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97    140597
         1.0       0.61      0.44      0.51      9403

    accuracy                           0.95    150000
   macro avg       0.79      0.71      0.74    150000
weighted avg       0.94      0.95      0.94    150000

